In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName('test').getOrCreate()

train = spark.read.csv('./train.csv', header='true', inferSchema='true')
train.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

Now we will explore the label columns.

In [2]:
from pyspark.sql.functions import kurtosis, skewness

train.select('SalePrice').describe().show()
train.agg(skewness('SalePrice')).show()
train.agg(kurtosis('SalePrice')).show()
# Last parameter is error tolerance
print(train.approxQuantile('SalePrice', [0.25, 0.50, 0.75], 0.05))
train.stat.freqItems(['SalePrice'], 0.1).show()

+-------+------------------+
|summary|         SalePrice|
+-------+------------------+
|  count|              1460|
|   mean|180921.19589041095|
| stddev| 79442.50288288663|
|    min|             34900|
|    max|            755000|
+-------+------------------+

+-------------------+
|skewness(SalePrice)|
+-------------------+
| 1.8809407460340333|
+-------------------+

+-------------------+
|kurtosis(SalePrice)|
+-------------------+
|  6.509812011089432|
+-------------------+

[127000.0, 157000.0, 205000.0]
+--------------------+
| SalePrice_freqItems|
+--------------------+
|[210000, 142125, ...|
+--------------------+



In [3]:
def findMissingValuesCols(df):
    numRows = df.count()
    nullCols = []
    for column in df.columns:
        c = df.filter(col(column).isNotNull()).count()
        if c != numRows:
            nullCols.append(c)
    return nullCols

missingCols = findMissingValuesCols(train)

In [4]:
from pyspark.ml.stat import ChiSquareTest

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

corrCols = [dtype[0] for dtype in train.dtypes if dtype[1] == 'int']
corrCols.remove('Id')

corrAssembler = VectorAssembler(inputCols=corrCols, outputCol='corrFeatures')
corrTrain = corrAssembler.transform(train)
corrMat = Correlation.corr(corrTrain, 'corrFeatures').head()

pdf = corrMat[0].toArray()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,12))

sns.heatmap(pdf, 
            xticklabels=corrCols,
            yticklabels=corrCols)

In [5]:
from pyspark.ml.feature import StringIndexer

def autoIndexer(df, maxDistinct):
    stringTypes = [dtype[0] for dtype in train.dtypes if dtype[1] == 'string']
    numRows = train.count()
    indexed = df
    indexedCols = []
    for column in stringTypes:
        distinctCount = train.select(column).distinct().count()
        if distinctCount < maxDistinct:
            indexedCol = 'Indexed' + column
            indexedCols.append(indexedCol)
            indexer = StringIndexer(inputCol=column, outputCol=indexedCol)
            indexed = indexer.fit(indexed).transform(indexed)
            indexed = indexed.drop(column)
    return indexedCols, indexed

In [6]:
from pyspark.ml.feature import OneHotEncoderEstimator, OneHotEncoder
from pyspark.ml import Pipeline

oneHotEncodedColumns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
                       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType',
                       'MiscFeature', ]
indexedOutputColumns = ['Indexed'+column for column in oneHotEncodedColumns]
oneHotEncodedOutputColumns = ['Encoded'+column for column in indexedOutputColumns]

indexers = [StringIndexer(inputCol=column, outputCol='Indexed'+column) for column in oneHotEncodedColumns]

oneHotEncoder = OneHotEncoderEstimator(inputCols=indexedOutputColumns, outputCols=oneHotEncodedOutputColumns)
# pipeline = Pipeline(stages=[*indexers, estimator])
# model = pipeline.fit(train)
# cleaned = model.transform(train)
# droppedColumns = [*oneHotEncodedColumns, *indexedOutputColumns]
# cleaned = cleaned.drop(*droppedColumns)
# cleaned.printSchema()

In [7]:
%%script false 

cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
       'HeatingQC', 'KitchenQual', 'Functional']

In [18]:
cols = [dtype[0] for dtype in train.dtypes if dtype[1]!='string']
cols.remove('SalePrice')

from pyspark.ml.feature import ChiSqSelector

chiSqAssembler = VectorAssembler(inputCols=cols, outputCol='features')
cleaned = chiSqAssembler.transform(train)
chiSqSelector = ChiSqSelector(numTopFeatures=10, featuresCol='features', outputCol='selectedFeatures', labelCol='SalePrice')
cleaned = chiSqSelector.fit(cleaned).transform(cleaned)
chiSqSelector.percentile

Param(parent='ChiSqSelector_4369b6e8a3cb4650e264', name='percentile', doc='Percentile of features that selector will select, ordered by ascending p-value.')

In [9]:
from pyspark.ml.regression import RandomForestRegressor

randomForest = RandomForestRegressor(featuresCol='selectedFeatures', labelCol='SalePrice', seed=42)


In [10]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

# grid = ParamGridBuilder().addGrid(randomForest.numTrees, [10, 15, 20]).build()

pipeline = Pipeline(stages=[*indexers, oneHotEncoder, chiSqAssembler, chiSqSelector, randomForest])

# crossval = CrossValidator(estimator=pipeline, 
#                           estimatorParamMaps=grid, 
#                           evaluator=RegressionEvaluator(labelCol="SalePrice"), 
#                           numFolds=10)

model = pipeline.fit(train)
df_r = model.transform(train)
evaluator=RegressionEvaluator(labelCol="SalePrice")
evaluator.evaluate(df_r,  {evaluator.metricName: "r2"})

0.8625773603318725

In [11]:
rfModel = model.stages[-1]

print('Number of features: ', rfModel.numFeatures)
print('Number of trees: ', rfModel.getNumTrees)
print('Feature Importances: ', rfModel.featureImportances)


Number of features:  10
Number of trees:  20
Feature Importances:  (10,[0,1,2,3,4,5,6,7,8,9],[0.0192426156999,0.459307192737,0.00972816499007,0.00905705388025,0.105944178937,0.06505743252,0.0317621143464,0.229945465731,0.0426214544914,0.0273343266662])


In [12]:
from pyspark.sql.types import IntegerType

test = spark.read.csv('./test.csv', header='true', inferSchema='true')
test = test.withColumn('BsmtFinSF1', test['BsmtFinSF1'].cast(IntegerType()))
test = test.withColumn('BsmtFinSF2', test['BsmtFinSF2'].cast(IntegerType()))
train.select('BsmtFinSF1').printSchema()
test.select('BsmtFinSF1').printSchema()

predictions = model.transform(test)
predictions

root
 |-- BsmtFinSF1: integer (nullable = true)

root
 |-- BsmtFinSF1: integer (nullable = true)



IllegalArgumentException: 'Data type StringType of column BsmtUnfSF is not supported.\nData type StringType of column TotalBsmtSF is not supported.\nData type StringType of column BsmtFullBath is not supported.\nData type StringType of column BsmtHalfBath is not supported.\nData type StringType of column GarageCars is not supported.\nData type StringType of column GarageArea is not supported.'

In [ ]:

cols = indexed.columns[:]
cols.remove('SalePrice')
cols.remove('Id')
print(cols)
assembler = VectorAssembler(inputCols=cols, outputCol='features')
assembled = assembler.transform(indexed)

In [ ]:
from pyspark.ml.stat import ChiSquareTest, Correlation

r = ChiSquareTest.test(assembled, 'features', 'SalePrice')
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

r.show()

In [ ]:
r1 = Correlation.corr(assembled, 'features')
r1.head(1)

In [ ]:
train.crosstab('OverallQual', 'OverallCond').show()

In [ ]:
# train.corr('OverallCond', 'OverallQual')
# train.dtypes
train.select('OverallCond').groupby('OverallCond')